# Important - activate GPU before starting session

- Menu > Settings > Accelerator > GPU T4 x2

# Outlines install

- breaking bug this week : if below fix doesn't work try installing older version such as `0.0.46`

In [2]:
!pip install outlines==0.0.46

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.5 MB/s eta 0:00:00:00:0100:01


In [3]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained( 
    MODEL_NAME,  
    device_map=device,  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

### Create LLM pipeline

In [5]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

## Define prompts

- define prompt template
- provide source document for "RAG" step
- specify query e.g. entities we want to extract

In [6]:
prompt_template = """Below is a document, followed by a query which can be answered by reading the document. Answer the query using only the original document. Return your answer in JSON format.

## DOCUMENT ##

{document}

## QUERY ##

{query}
"""

In [7]:
document_orange = """Orange S.A. (formerly France Télécom S.A., stylised as france telecom) is a French multinational telecommunications corporation headquartered in Issy-les-Moulineaux, near Paris.

Orange has been the corporation's main brand for mobile, landline, internet and Internet Protocol television (IPTV) services since 2006. It traces its origins back to Hutchison Whampoa acquiring a controlling stake in Microtel Communications in 1994 in the United Kingdom. Microtel Communications became a subsidiary of Mannesmann in 1999 and then was acquired by France Télécom in 2000. The former French public monopoly thus became internationalized following this takeover and has pursued an expansionist policy since. The group is now present in Europe, Africa and the French West Indies. Since February 2012, as a result of the company's decision to transfer its fixed-line telephony activities being to its Orange brand, all offers marketed by France Télécom are Orange-branded; and on July 1, 2013, France Télécom itself was rebranded after it. In 2019, Orange S.A. employed nearly 148,000 people worldwide, including 88,000 in France.

The corporation has throughout the years expanded its activities to include content sales (music, cinema, downloads, etc.), e-commerce, online advertising, M2M, home automation and remote assistance solutions. In 2023, it generated over €43 billion in revenue across all its businesses."""

In [8]:
query_companies = "Return a JSON of the companies found in the document."

## Generate answer with LLM

In [9]:
prompt = prompt_template.format(document=document_orange,
                                query=query_companies)

messages = [ 
    {"role": "system", "content": "You are helpful asssistant that finds answers to questions and queries in documents."}, 
    {"role": "user", "content": prompt}, 
] 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
}

In [10]:
output = llm(messages, **generation_args)

result_no_struct = output[0]['generated_text']

result_no_struct

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


' ```json\n\n{\n\n  "companies": [\n\n    {\n\n      "name": "Orange S.A.",\n\n      "former_names": [\n\n        "France Télécom S.A.",\n\n        "France Télécom"\n\n      ],\n\n      "stylised_name": "France Telecom",\n\n      "headquarters": "Issy-les-Moulineaux, near Paris",\n\n      "industry": "telecommunications",\n\n      "services": [\n\n        "mobile",\n\n        "landline",\n\n        "internet",\n\n        "Internet Protocol television (IPTV)"\n\n      ],\n\n      "history": {\n\n        "origin": "Hutchison Whampoa acquired a controlling stake in Microtel Communications in 1994 in the United Kingdom.",\n\n        "subsidiary_changes": [\n\n          {\n\n            "year": 1999,\n\n            "event": "Microtel Communications became a subsidiary of Mannesmann."\n\n          },\n\n          {\n\n            "year": 2000,\n\n            "event": "Mannesmann was acquired by France Télécom."\n\n          }\n\n        ],\n\n        "expansion": "The former French public mo

In [11]:
print(result_no_struct)

 ```json

{

  "companies": [

    {

      "name": "Orange S.A.",

      "former_names": [

        "France Télécom S.A.",

        "France Télécom"

      ],

      "stylised_name": "France Telecom",

      "headquarters": "Issy-les-Moulineaux, near Paris",

      "industry": "telecommunications",

      "services": [

        "mobile",

        "landline",

        "internet",

        "Internet Protocol television (IPTV)"

      ],

      "history": {

        "origin": "Hutchison Whampoa acquired a controlling stake in Microtel Communications in 1994 in the United Kingdom.",

        "subsidiary_changes": [

          {

            "year": 1999,

            "event": "Microtel Communications became a subsidiary of Mannesmann."

          },

          {

            "year": 2000,

            "event": "Mannesmann was acquired by France Télécom."

          }

        ],

        "expansion": "The former French public monopoly became internationalized following the takeover and ha

# Repeat with Outlines

- last minute bug with current version, installed old one instead : [https://github.com/dottxt-ai/outlines/issues/1198](https://github.com/dottxt-ai/outlines/issues/1198)

In [12]:
from outlines.integrations.transformers import JSONPrefixAllowedTokens

In [14]:
from pydantic import BaseModel, Field
from typing import List, Optional

## Define custom schema

In [17]:
class Company(BaseModel):
    company_name: str = Field(..., description="The name of a company or of a business")
    former_company_names: Optional[List[str]] = Field(..., description="List of any names the company may have used previously or that were acquired or merged")

class CompanyExtractor(BaseModel):
    companies_in_document: List[Company]

prefix_allowed_tokens_fn = JSONPrefixAllowedTokens(
    schema=CompanyExtractor,
    tokenizer_or_pipe=llm, # our previously defined LLM pipeline
)

In [18]:
structured_generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0,
    "prefix_allowed_tokens_fn": prefix_allowed_tokens_fn,
}

structured_output = llm(messages, **structured_generation_args)

structured_output

[{'generated_text': '{ "companies_in_document": [ { "company_name": "Orange S.A.", "former_company_names": ["France Télécom S.A.", "Microtel Communications", "Mannesmann"] }, { "company_name": "Hutchison Whampoa", "former_company_names": [] }, { "company_name": "Mannesmann", "former_company_names": [] } ] }'}]

In [19]:
result_with_struct = structured_output[0]["generated_text"]

result_with_struct

'{ "companies_in_document": [ { "company_name": "Orange S.A.", "former_company_names": ["France Télécom S.A.", "Microtel Communications", "Mannesmann"] }, { "company_name": "Hutchison Whampoa", "former_company_names": [] }, { "company_name": "Mannesmann", "former_company_names": [] } ] }'

In [20]:
import json

test_loads_as_json = json.loads(result_with_struct)

test_loads_as_json

{'companies_in_document': [{'company_name': 'Orange S.A.',
   'former_company_names': ['France Télécom S.A.',
    'Microtel Communications',
    'Mannesmann']},
  {'company_name': 'Hutchison Whampoa', 'former_company_names': []},
  {'company_name': 'Mannesmann', 'former_company_names': []}]}